In [13]:
import pandas as pd

file = r'/Users/jenny.lin/CausalImpact_Time_Series_on_Features/data/MLS_Data_Change_Properties_Limited.csv'

In [14]:
df = pd.read_csv(file)

In [15]:
df.columns

Index(['CC_PROPERTY_ID', 'SITUSSTATE', 'SITUSCITY', 'SITUSZIP5',
       'FIRST_LISTING_DATE', 'FIRST_BEDROOMS', 'FIRST_FULL_BATHS',
       'FIRST_HALF_BATHS', 'FIRST_SQFT', 'LAST_LISTING_DATE', 'LAST_BEDROOMS',
       'LAST_FULL_BATHS', 'LAST_HALF_BATHS', 'LAST_SQFT', 'BEDROOM_CHANGE',
       'FULL_BATH_CHANGE', 'HALF_BATH_CHANGE', 'SQFT_CHANGE',
       'DAYS_BETWEEN_SNAPSHOTS', 'YEAR_BUILT', 'LOTSIZESQFT'],
      dtype='object')

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from causalimpact import CausalImpact
from datetime import datetime

# Normalize column names to handle case differences
df.columns = df.columns.str.upper()

print("Columns found:", df.columns.tolist())
print("Shape:", df.shape)

# Check what we have
print("\nUnique property types:", df['PROPERTY_TYPE'].unique() if 'PROPERTY_TYPE' in df.columns else 'N/A')
print("Date range:", df['RECORD_DATE_TIME'].min() if 'RECORD_DATE_TIME' in df.columns else 'N/A',
      "to", df['RECORD_DATE_TIME'].max() if 'RECORD_DATE_TIME' in df.columns else 'N/A')

# Convert date column
df['RECORD_DATE_TIME'] = pd.to_datetime(df['RECORD_DATE_TIME'])

# Check for treated properties
if 'PROPERTY_TYPE' not in df.columns or df[df['PROPERTY_TYPE'] == 'TREATED'].empty:
    print("\nERROR: No treated properties found in the data!")
    print("Available property types:", df['PROPERTY_TYPE'].unique() if 'PROPERTY_TYPE' in df.columns else 'Column not found')
    print("\nShowing first few rows:")
    print(df.head(20))
else:
    # Focus on one treated property for demonstration
    treated_property_id = df[df['PROPERTY_TYPE'] == 'TREATED']['CC_PROPERTY_ID'].iloc[0]
    treated_data = df[df['CC_PROPERTY_ID'] == treated_property_id].copy()

    print(f"\nAnalyzing property: {treated_property_id}")
    print(f"Number of observations: {len(treated_data)}")

    # Find the intervention point (when features changed)
    if 'TREATMENT_PERIOD' in treated_data.columns:
        intervention_date = treated_data[treated_data['TREATMENT_PERIOD'] == 'POST_TREATMENT']['RECORD_DATE_TIME'].min()
        print(f"Intervention Date: {intervention_date}")

        # Show before/after feature values
        pre_features = treated_data[treated_data['TREATMENT_PERIOD'] == 'PRE_TREATMENT'][['BEDROOMS', 'FULL_BATHS', 'PRICE']].iloc[0]
        post_features = treated_data[treated_data['TREATMENT_PERIOD'] == 'POST_TREATMENT'][['BEDROOMS', 'FULL_BATHS', 'PRICE']].iloc[0]

        print(f"\nPre-treatment: {pre_features['BEDROOMS']} bed, {pre_features['FULL_BATHS']} bath, ${pre_features['PRICE']:,.0f}")
        print(f"Post-treatment: {post_features['BEDROOMS']} bed, {post_features['FULL_BATHS']} bath, ${post_features['PRICE']:,.0f}")
    else:
        print("WARNING: TREATMENT_PERIOD column not found")

Columns found: ['CC_PROPERTY_ID', 'SITUSSTATE', 'SITUSCITY', 'SITUSZIP5', 'FIRST_LISTING_DATE', 'FIRST_BEDROOMS', 'FIRST_FULL_BATHS', 'FIRST_HALF_BATHS', 'FIRST_SQFT', 'LAST_LISTING_DATE', 'LAST_BEDROOMS', 'LAST_FULL_BATHS', 'LAST_HALF_BATHS', 'LAST_SQFT', 'BEDROOM_CHANGE', 'FULL_BATH_CHANGE', 'HALF_BATH_CHANGE', 'SQFT_CHANGE', 'DAYS_BETWEEN_SNAPSHOTS', 'YEAR_BUILT', 'LOTSIZESQFT']
Shape: (105, 21)

Unique property types: N/A
Date range: N/A to N/A


KeyError: 'RECORD_DATE_TIME'